# Setup training framework

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def train_model_and_plot_stats(
        model, error, learning_rule, train_data, scheduler,
        valid_data, num_epochs, stats_interval, 
        notebook=True):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, scheduler, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for 5 epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [2]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102018 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

KeysView(<numpy.lib.npyio.NpzFile object at 0x10d6eefd0>)
KeysView(<numpy.lib.npyio.NpzFile object at 0x10d6ee908>)


In [3]:
from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import AdamLearningRule, GradientDescentLearningRule, RMSPropLearningRule
from mlp.optimisers import Optimiser
from mlp.schedulers import CosineAnnealingWithWarmRestarts

# Apply CosineAnnealingWithWarmRestarts to SGD

In [ ]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

#setup hyperparameters
min_learning_rate = 1
max_learning_rate = 0.001
total_iters_per_period = 25
max_learning_rate_discount_factor = 0.8
period_iteration_expansion_factor = 3
num_epochs = 100

stats_interval = 1
input_dim, output_dim, hidden_dim = 784, 47, 100

weights_init = GlorotUniformInit(rng=rng)
biases_init = ConstantInit(0.)
model = MultipleLayerModel([
    AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
    ReluLayer(),
    AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init), 
    ReluLayer(),
    AffineLayer(hidden_dim, output_dim, weights_init, biases_init)
])

error = CrossEntropySoftmaxError()
# Use a basic gradient descent learning rule
learning_rule = GradientDescentLearningRule()

scheduler = CosineAnnealingWithWarmRestarts(min_learning_rate, max_learning_rate, total_iters_per_period, max_learning_rate_discount_factor,
                 period_iteration_expansion_factor)

#Remember to use notebook=False when you write a script to be run in a terminal
_ = train_model_and_plot_stats(
    model, error, learning_rule, train_data, scheduler, valid_data, num_epochs, stats_interval, notebook=True)

Epoch 1: 5.9s to complete
    error(train)=3.80e+00, acc(train)=3.81e-02, error(valid)=3.80e+00, acc(valid)=3.94e-02


Epoch 2: 4.6s to complete
    error(train)=3.72e+00, acc(train)=7.36e-02, error(valid)=3.72e+00, acc(valid)=7.58e-02


Epoch 3: 4.7s to complete
    error(train)=3.62e+00, acc(train)=1.25e-01, error(valid)=3.62e+00, acc(valid)=1.25e-01


Epoch 4: 4.5s to complete
    error(train)=3.48e+00, acc(train)=1.75e-01, error(valid)=3.48e+00, acc(valid)=1.78e-01


Epoch 5: 4.5s to complete
    error(train)=3.30e+00, acc(train)=2.32e-01, error(valid)=3.30e+00, acc(valid)=2.33e-01


...

Epoch 19: 4.8s to complete
    error(train)=1.53e+00, acc(train)=5.85e-01, error(valid)=1.53e+00, acc(valid)=5.79e-01


Epoch 20: 4.4s to complete
    error(train)=1.50e+00, acc(train)=5.92e-01, error(valid)=1.50e+00, acc(valid)=5.86e-01


Epoch 21: 4.4s to complete
    error(train)=1.48e+00, acc(train)=5.98e-01, error(valid)=1.48e+00, acc(valid)=5.91e-01


Epoch 22: 4.3s to complete
    error(train)=1.45e+00, acc(train)=6.02e-01, error(valid)=1.45e+00, acc(valid)=5.99e-01


Epoch 23: 4.2s to complete
    error(train)=1.43e+00, acc(train)=6.08e-01, error(valid)=1.43e+00, acc(valid)=6.01e-01
